In [1]:
import seaborn as sns
import os
from scipy.stats import kurtosis, skew

from rcv_distribution import *
from MDS_analysis import *
from voting_rules import *
import random
from itertools import permutations
from collections import defaultdict
from collections import Counter

In [4]:
directory = "dataverse_files"
table = pd.read_csv("permissive.csv")
for filename in os.listdir(directory):
    try:
        csv = os.path.join(directory, filename)
        ballots, candidates = parse_election_data(csv)

        null_directory = "null_elections"
        
        df = pd.read_csv(os.path.join(null_directory, filename))
        normalized_distances = pd.Series(df['position'].values, index=df['candidate']).to_dict()

        consistent_ballots, gamma = get_permissive_gamma(ballots, normalized_distances)

        election = voting_rules(ballots, candidates)
        consistent_election = voting_rules(consistent_ballots, candidates)

        irv1 = election.irv()
        irv2 = consistent_election.irv()

        if (irv1 != irv2):
            table.loc[table["filename"] == filename, "linear voters failure"] = "yes"
            print(filename, ": actual election: ", irv1, " consistent election: ", irv2)
        else:
            table.loc[table["filename"] == filename, "linear voters failure"] = "no"
            print("same winner: ", filename, ": actual election: ", irv1, " consistent election: ", irv2)
    except Exception as e:
        # Print the summary of the error
        print(f"An error occurred: {e}")
        
table.to_csv("diff.csv")

same winner:  Alaska_04102020_PRESIDENTOFTHEUNITEDSTATES.csv : actual election:  Joseph R. Biden  consistent election:  Joseph R. Biden
same winner:  Alaska_08162022_HouseofRepresentativesSpecial.csv : actual election:  Peltola, Mary S.  consistent election:  Peltola, Mary S.
same winner:  Alaska_11082022_GovernorLieutenantGovernor.csv : actual election:  Dunleavy/Dahlstrom  consistent election:  Dunleavy/Dahlstrom
same winner:  Alaska_11082022_HouseDistrict1.csv : actual election:  Ortiz, Daniel H. "Dan"  consistent election:  Ortiz, Daniel H. "Dan"
same winner:  Alaska_11082022_HouseDistrict10.csv : actual election:  Johnson, Craig W.  consistent election:  Johnson, Craig W.
same winner:  Alaska_11082022_HouseDistrict11.csv : actual election:  Coulombe, Julie  consistent election:  Coulombe, Julie
same winner:  Alaska_11082022_HouseDistrict12.csv : actual election:  Schrage, Calvin R.  consistent election:  Schrage, Calvin R.
same winner:  Alaska_11082022_HouseDistrict13.csv : actual

In [9]:
table[table["linear voters failure"]=='yes']

,filename,diff,linear voters failure,gamma,#candidates,level,partisan,type,IRV1,IRV2,Condorcet1,Plurality1,Approval1IRV2,Condorcet2,Plurality2,Approval2,Approval1
134,Corvallis_11082022_Mayor.csv,"['IRV', 'Condorcet', 'Plurality']",yes,0.806261,3,LOCAL,NO,E,Charles Maughan,Andrew Struthers,Charles Maughan,Charles Maughan,NaN,Andrew Struthers,Andrew Struthers,Andrew Struthers,Andrew Struthers
137,Maine_07142020_DemocraticCandidateforMaineStat...,"['IRV', 'Condorcet', 'Plurality', 'Approval']",yes,0.811807,3,STATE,DP,R,"Zager, Samuel Lewis","Grant, Benjamin K.","Zager, Samuel Lewis","Zager, Samuel Lewis",NaN,"Grant, Benjamin K.","Grant, Benjamin K.","Grant, Benjamin K.","Zager, Samuel Lewis"
139,Maine_07142020_DemocraticCandidateforMaineStat...,"['IRV', 'Condorcet', 'Approval']",yes,0.776243,3,STATE,DP,R,"Arford, Poppy","Perreault, Corinne Ann","Arford, Poppy","Arford, Poppy",NaN,"Perreault, Corinne Ann","Arford, Poppy","Perreault, Corinne Ann","Arford, Poppy"
140,Maine_07142020_DemocraticCandidateforMaineStat...,"['IRV', 'Condorcet', 'Plurality', 'Approval']",yes,0.800394,3,STATE,DP,R,"Crafts, Lydia V.","Pieh, Wendy","Crafts, Lydia V.","Crafts, Lydia V.",NaN,"Pieh, Wendy","Pieh, Wendy","Pieh, Wendy","Crafts, Lydia V."
141,Maine_07142020_DemocraticCandidateforMaineStat...,"['IRV', 'Condorcet', 'Plurality', 'Approval']",yes,0.781368,3,STATE,DP,R,"Curry, Glenn Chip","Stanicki, Robyn M.","Curry, Glenn Chip","Curry, Glenn Chip",NaN,"Stanicki, Robyn M.","Stanicki, Robyn M.","Stanicki, Robyn M.","Curry, Glenn Chip"
146,Minneapolis_11022021_ParkBoardDistrict2.csv,"['IRV', 'Condorcet', 'Approval']",yes,0.854551,3,LOCAL,NO,B,Becka Thompson,Eric Moran,Becka Thompson,Becka Thompson,NaN,Eric Moran,Becka Thompson,Eric Moran,Becka Thompson
158,Minneapolis_11072017_Ward1CityCouncil.csv,"['IRV', 'Condorcet', 'Plurality', 'Approval']",yes,0.935236,3,LOCAL,NO,R,Kevin Reich,Jillia Pessenda,Kevin Reich,Kevin Reich,NaN,Jillia Pessenda,Jillia Pessenda,Jillia Pessenda,Kevin Reich
159,NewYorkCity_06222021_DEMBoroughPresidentQueens...,"['IRV', 'Condorcet', 'Plurality']",yes,0.922902,3,LOCAL,DP,E,Donovan J. Richards Jr.,Elizabeth S. Crowley,Donovan J. Richards Jr.,Donovan J. Richards Jr.,NaN,Elizabeth S. Crowley,Elizabeth S. Crowley,Elizabeth S. Crowley,Elizabeth S. Crowley
166,Oakland_11032020_MemberCityCouncilAtLargeOakla...,"['IRV', 'Condorcet', 'Plurality', 'Approval']",yes,0.877497,3,LOCAL,NO,R,REBECCA KAPLAN,DERRECK B. JOHNSON,REBECCA KAPLAN,REBECCA KAPLAN,NaN,DERRECK B. JOHNSON,DERRECK B. JOHNSON,DERRECK B. JOHNSON,REBECCA KAPLAN
168,Oakland_11032020_MemberCityCouncilDist5Oakland...,"['IRV', 'Condorcet', 'Approval']",yes,0.852951,3,LOCAL,NO,R,NOEL GALLO,RICHARD SANTOS RAYA,NOEL GALLO,NOEL GALLO,NaN,RICHARD SANTOS RAYA,NOEL GALLO,RICHARD SANTOS RAYA,NOEL GALLO


In [2]:
def generate_permutations(items, k):
    all_permutations = []
    
    for length in range(1, k + 1):
        perms = permutations(items, length)
        all_permutations.extend(perms)
    
    return all_permutations

In [3]:
def generate_and_group_permutations(items, choices):
    n = len(items)
    grouped_permutations = defaultdict(list)
    
    for i in range(1, choices + 1):
        for perm in permutations(items, i):
            grouped_permutations[perm[0]].append(perm)
    
    return grouped_permutations

In [4]:
# This function is keeping the first place choices of the inconsistent voters.
def generate_inconsistent_ballots(ballots, candidates, filename, runs):
    directory = "null_elections"
    csv = os.path.join(directory, filename) 
    df = pd.read_csv(csv)  
    candidates = df['candidate'].values
    election = pd.read_csv("election_table.csv")
    choices = min(len(candidates), round(election.loc[election['filename']==filename, 'choices'].values[0]))
    if choices == 1:
        choices = len(candidates)
    print(choices)
    normalized_distances = pd.Series(df['position'].values, index=df['candidate']).to_dict()
    print(normalized_distances)
    #plot_KDE(ballots, normalized_distances)
    consistent_ballots, gamma = get_permissive_gamma(ballots, normalized_distances)

    all_voters = 0
    for b in ballots:
        if len(b) > 0:
            all_voters += ballots[b]
    consistent_voters = sum(consistent_ballots.values())
    inconsistent_voters = all_voters - consistent_voters
    #print(inconsistent_voters)

    
    grouped_permutations = generate_and_group_permutations(candidates, choices)
    
    for candidate, perms in grouped_permutations.items():
        grouped_permutations[candidate] = [perm for perm in perms if (perm not in consistent_ballots) and is_consistent(perm, normalized_distances)==False]
        """print(grouped_permutations[c])
        for ranking in grouped_permutations[c]:
            if ranking in consistent_ballots:
                print(ranking)
                grouped_permutations[c].remove(ranking)
            else:
                print(ranking, " was not in consistent ballots")"""
    #print(grouped_permutations)

    inconsistent_ballots = {}
    for b in ballots:
        if len(b) > 0 and b not in consistent_ballots:
            if b not in inconsistent_ballots:
                inconsistent_ballots[b] = 0
            inconsistent_ballots[b] += ballots[b]


    candidates_freq = {}
    for b in inconsistent_ballots:
        candidate = b[0]
        if candidate not in candidates_freq:
            candidates_freq[candidate] = 0
        candidates_freq[candidate] += inconsistent_ballots[b]
    #print(candidates_freq)

    random_inconsistent_ballots = {} 

    for i in range(runs):
        for candidate in candidates_freq:
            first_place = candidates_freq[candidate]
            if first_place > 0:
               
                random_ballots = random.choices(grouped_permutations[candidate], k=first_place)
                for b in random_ballots:
                    if b not in random_inconsistent_ballots:
                        random_inconsistent_ballots[b] = 0
                    random_inconsistent_ballots[b] += 1
    return random_inconsistent_ballots, consistent_ballots



In [5]:
# This function is generating totaly random ballots instead of the inconsistent ones (not keeping the first place choices)

def generate_random_inconsistent_ballots(ballots, candidates, filename):
    election = pd.read_csv("election_table.csv")
    choices = min(len(candidates), round(election.loc[election['filename']==filename, 'choices'].values[0]))
    if choices == 1:
        choices = len(candidates)

    print(choices)
    directory = "null_elections"
    csv = os.path.join(directory, filename) 
    df = pd.read_csv(csv)

    perms = generate_permutations(candidates, choices)

    normalized_distances = pd.Series(df['position'].values, index=df['candidate']).to_dict()
    print(normalized_distances)
    consistent_ballots, gamma = get_permissive_gamma(ballots, normalized_distances)

    perms = [perm for perm in perms if (perm not in consistent_ballots) and is_consistent(perm, normalized_distances)==False]    
    
   
    all_voters = 0
    for b in ballots:
        if len(b) > 0:
            all_voters += ballots[b]
    consistent_voters = sum(consistent_ballots.values())
    inconsistent_voters = all_voters - consistent_voters
    inconsistent_ballots_list = random.choices(perms, k=inconsistent_voters)
    inconsistent_ballots = {}

    for ballot in inconsistent_ballots_list:
        if ballot not in inconsistent_ballots:
            inconsistent_ballots[ballot] = 0
        inconsistent_ballots[ballot] += 1

    return inconsistent_ballots, consistent_ballots
    

In [6]:
filename = "Oakland_11082022_Mayor.csv"
ballots, candidates = parse_election_data(os.path.join("dataverse_files", filename))
print(candidates)

['SHENG THAO', 'LOREN MANUEL TAYLOR', 'JOHN REIMANN', 'ALLYSSA VICTORY VILLANUEVA', 'TREVA D. REID', 'SENECA SCOTT', 'PETER Y. LIU', 'IGNACIO DE LA FUENTE', 'GREGORY HODGE', 'TYRON C. JORDAN']


In [7]:
freqs = get_frequency(ballots, candidates)
print(freqs)

{'SHENG THAO': 0.3154222459155195, 'LOREN MANUEL TAYLOR': 0.32802847376838407, 'IGNACIO DE LA FUENTE': 0.10334284549531186, 'TREVA D. REID': 0.06136912414939321, 'ALLYSSA VICTORY VILLANUEVA': 0.08742042710652576, 'SENECA SCOTT': 0.030504249114114586, 'GREGORY HODGE': 0.04715575903916711, 'TYRON C. JORDAN': 0.0074869077111229575, 'PETER Y. LIU': 0.008560945780676722, 'JOHN REIMANN': 0.010709021919784251}


In [ ]:
ignore = [] 
for c in freqs:
    if freq[c] 

In [13]:
inconsistent_ballots, consistent_ballots = generate_random_inconsistent_ballots(ballots, candidates, filename)

TypeError: 'str' object is not callable

In [14]:
inconsistent_ballots, consistent_ballots = generate_random_inconsistent_ballots(ballots, candidates, filename)
random_ballots = {**consistent_ballots, **inconsistent_ballots}

random_elction = voting_rules(random_ballots, candidates)
winner_random = random_elction.irv()
print("random: ", winner_random[0])
print("symmary:")
for r in winner_random[1]:
    print(r, "  ", winner_random[1][r])

election = voting_rules(ballots, candidates)
winner_real = election.irv()
print("real: ", winner_real[0])
print("symmary:")
for r in winner_real[1]:
    print(r, "  ", winner_real[1][r])

consistent_election = voting_rules(consistent_ballots, candidates)
winner_consistent = consistent_election.irv()
print("consistent: ", winner_consistent[0])

for r in winner_consistent[1]:
    print(r, "  ", winner_consistent[1][r])


5
{'SHENG THAO': 3.221462336801974, 'LOREN MANUEL TAYLOR': 4.303122506518532, 'JOHN REIMANN': 7.760561435492615, 'ALLYSSA VICTORY VILLANUEVA': 5.0497656581372485, 'TREVA D. REID': 3.935815220320551, 'SENECA SCOTT': 4.650772364000745, 'PETER Y. LIU': 9.0, 'IGNACIO DE LA FUENTE': 3.397828546854356, 'GREGORY HODGE': 4.711911133144116, 'TYRON C. JORDAN': 0.0}
random:  LOREN MANUEL TAYLOR
symmary:
round_1    {'SHENG THAO': 23499, 'LOREN MANUEL TAYLOR': 31276, 'TREVA D. REID': 10609, 'IGNACIO DE LA FUENTE': 13623, 'ALLYSSA VICTORY VILLANUEVA': 9908, 'GREGORY HODGE': 9128, 'SENECA SCOTT': 8078, 'TYRON C. JORDAN': 7003, 'PETER Y. LIU': 7460, 'JOHN REIMANN': 6972}
round_2    {'SHENG THAO': 24334, 'LOREN MANUEL TAYLOR': 31985, 'TREVA D. REID': 11419, 'IGNACIO DE LA FUENTE': 14411, 'ALLYSSA VICTORY VILLANUEVA': 10626, 'GREGORY HODGE': 9787, 'SENECA SCOTT': 8867, 'TYRON C. JORDAN': 7761, 'PETER Y. LIU': 8130}
round_3    {'SHENG THAO': 25181, 'LOREN MANUEL TAYLOR': 32988, 'TREVA D. REID': 12358, 'I

In [5]:

random_inconsistent_ballots, consistent_ballots = generate_inconsistent_ballots(ballots, candidates, filename, runs=1)
print(random_inconsistent_ballots)

{('SHENG THAO', 'TYRON C. JORDAN', 'IGNACIO DE LA FUENTE', 'SENECA SCOTT', 'LOREN MANUEL TAYLOR'): 12, ('SHENG THAO', 'TYRON C. JORDAN', 'GREGORY HODGE', 'SENECA SCOTT'): 7, ('SHENG THAO', 'TREVA D. REID', 'LOREN MANUEL TAYLOR', 'IGNACIO DE LA FUENTE', 'GREGORY HODGE'): 11, ('SHENG THAO', 'TYRON C. JORDAN', 'JOHN REIMANN', 'TREVA D. REID', 'SENECA SCOTT'): 7, ('SHENG THAO', 'PETER Y. LIU', 'JOHN REIMANN', 'ALLYSSA VICTORY VILLANUEVA', 'TYRON C. JORDAN'): 13, ('SHENG THAO', 'GREGORY HODGE', 'SENECA SCOTT', 'LOREN MANUEL TAYLOR', 'TYRON C. JORDAN'): 14, ('SHENG THAO', 'SENECA SCOTT', 'TYRON C. JORDAN', 'TREVA D. REID', 'ALLYSSA VICTORY VILLANUEVA'): 6, ('SHENG THAO', 'LOREN MANUEL TAYLOR', 'SENECA SCOTT', 'TYRON C. JORDAN', 'TREVA D. REID'): 7, ('SHENG THAO', 'LOREN MANUEL TAYLOR', 'IGNACIO DE LA FUENTE', 'ALLYSSA VICTORY VILLANUEVA', 'JOHN REIMANN'): 8, ('SHENG THAO', 'IGNACIO DE LA FUENTE', 'SENECA SCOTT', 'LOREN MANUEL TAYLOR', 'GREGORY HODGE'): 8, ('SHENG THAO', 'TREVA D. REID', 'IGN

In [36]:
runs = 1
count = 0
for i in range(runs):
    random_inconsistent_ballots, consistent_ballots = generate_random_inconsistent_ballots(ballots, candidates, filename)

    random_ballots = {**consistent_ballots, **random_inconsistent_ballots}

    print("generated ballots")
    for b in random_ballots:
        if b in consistent_ballots:
            c = "consistent"
        else:
            c = "inconsistent"
        print(b, " ", random_ballots[b], c)
    
    random_elction = voting_rules(random_ballots, candidates)
    print("random: ", random_elction.irv())

    print("real ballots")
    for b in ballots:
        print(b, " " , ballots[b])
    election = voting_rules(ballots, candidates)
    print("real: ", election.irv())

    consistent_election = voting_rules(consistent_ballots, candidates)
    print("consistent: ", consistent_election.irv())

    print("consistent ballots")
    for b in consistent_ballots:
        print(b, " ", consistent_ballots[b])
    for b in random_ballots:
        if b not in consistent_ballots:
            print(b, " ", random_ballots[b])
    

    if random_elction.irv() == election.irv():
        count += 1
    print()
print(count)

3
{'Cam Gordon': 3.001461293568189, 'Robin Wonsley Worlobah': 4.0, 'Yusra Arab': 2.6285290356824835, 'Tom Anderson': 2.3619814063116333, 'Guy T Gaskin': 0.0}
generated ballots
('Cam Gordon', 'Robin Wonsley Worlobah', 'Yusra Arab')   334 consistent
('Yusra Arab', 'Tom Anderson')   358 consistent
('Yusra Arab', 'Tom Anderson', 'Cam Gordon')   257 consistent
('Yusra Arab',)   802 consistent
('Cam Gordon', 'Robin Wonsley Worlobah')   596 consistent
('Yusra Arab', 'Tom Anderson', 'Robin Wonsley Worlobah')   196 consistent
('Robin Wonsley Worlobah', 'Cam Gordon')   643 consistent
('Cam Gordon', 'Yusra Arab')   211 consistent
('Cam Gordon', 'Robin Wonsley Worlobah', 'Tom Anderson')   130 consistent
('Guy T Gaskin',)   302 consistent
('Robin Wonsley Worlobah', 'Yusra Arab')   261 consistent
('Robin Wonsley Worlobah',)   405 consistent
('Cam Gordon',)   484 consistent
('Tom Anderson', 'Yusra Arab')   211 consistent
('Yusra Arab', 'Cam Gordon')   263 consistent
('Yusra Arab', 'Cam Gordon', 'Tom 

In [123]:
print(sum(random_inconsistent_ballots.values()))
for b in random_inconsistent_ballots:
    if len(b) > 0:
        print(b, ": ", random_inconsistent_ballots[b])

41124
('Bernardo Tuma', 'Dale Washam', 'Jan Shabro') :  614
('Bernardo Tuma', 'Dale Washam', 'Terry Lee') :  669
('Bernardo Tuma', 'Jan Shabro', 'Barbara Gelman') :  657
('Bernardo Tuma', 'Terry Lee', 'Beverly Davidson') :  670
('Bernardo Tuma', 'Dale Washam', 'Beverly Davidson') :  628
('Bernardo Tuma', 'Jan Shabro', 'Beverly Davidson') :  646
('Bernardo Tuma', 'Barbara Gelman', 'Beverly Davidson') :  621
('Bernardo Tuma', 'Terry Lee', 'Barbara Gelman') :  616
('Bernardo Tuma', 'Terry Lee', 'Jan Shabro') :  666
('Bernardo Tuma', 'Dale Washam', 'Barbara Gelman') :  622
('Beverly Davidson', 'Terry Lee', 'Jan Shabro') :  782
('Beverly Davidson', 'Dale Washam', 'Jan Shabro') :  799
('Beverly Davidson', 'Terry Lee', 'Barbara Gelman') :  814
('Beverly Davidson', 'Dale Washam', 'Barbara Gelman') :  859
('Beverly Davidson', 'Jan Shabro', 'Barbara Gelman') :  824
('Beverly Davidson', 'Dale Washam', 'Terry Lee') :  844
('Jan Shabro', 'Bernardo Tuma', 'Beverly Davidson') :  1234
('Jan Shabro', '

In [128]:
random_ballots = {**consistent_ballots, **random_inconsistent_ballots}
random_elction = voting_rules(random_ballots, candidates)
print("random: ", random_elction.irv())

election = voting_rules(ballots, candidates)
print("real: ", election.irv())

consistent_election = voting_rules(consistent_ballots, candidates)
print("consistent: ", consistent_election.irv())

random:  Dale Washam
real:  Dale Washam
consistent:  Dale Washam


In [55]:
for b in consistent_ballots:
    print(b)

('Peltola, Mary S.', 'Begich, Nick')
('Peltola, Mary S.',)
('Peltola, Mary S.', 'Begich, Nick', 'Palin, Sarah')
('Palin, Sarah',)
('Begich, Nick', 'Palin, Sarah', 'Peltola, Mary S.')
('Begich, Nick',)
('Begich, Nick', 'Peltola, Mary S.', 'Palin, Sarah')
('Palin, Sarah', 'Begich, Nick')
('Palin, Sarah', 'Begich, Nick', 'Peltola, Mary S.')
('Begich, Nick', 'Peltola, Mary S.')
('Begich, Nick', 'Palin, Sarah')
('Peltola, Mary S.', 'Palin, Sarah')
('Palin, Sarah', 'Peltola, Mary S.')


In [99]:
for b in ballots:
    print(b, ": ", ballots[b])

('Hinda Miller', 'Kevin Curley', 'Louie the Cowman Beaudin') :  10
('Bob Kiss', 'Hinda Miller', 'Loyal Ploof') :  86
('Hinda Miller', 'Bob Kiss') :  886
('Bob Kiss', 'Hinda Miller') :  1142
('Bob Kiss', 'Hinda Miller', 'Louie the Cowman Beaudin') :  77
('Loyal Ploof', 'Louie the Cowman Beaudin', 'Kevin Curley', 'Hinda Miller', 'Bob Kiss') :  1
('Bob Kiss', 'Hinda Miller', 'Kevin Curley') :  254
('Hinda Miller', 'Bob Kiss', 'Louie the Cowman Beaudin', 'Kevin Curley', 'Loyal Ploof') :  22
('Hinda Miller',) :  551
('Bob Kiss', 'Kevin Curley', 'Hinda Miller', 'Loyal Ploof', 'Louie the Cowman Beaudin') :  75
('Hinda Miller', 'Bob Kiss', 'Kevin Curley') :  202
('Bob Kiss',) :  495
('Hinda Miller', 'Bob Kiss', 'Kevin Curley', 'Loyal Ploof', 'Louie the Cowman Beaudin') :  96
('Kevin Curley',) :  802
('Bob Kiss', 'Loyal Ploof', 'Hinda Miller') :  25
('Kevin Curley', 'Louie the Cowman Beaudin', 'Loyal Ploof') :  35
('Hinda Miller', 'Bob Kiss', 'Kevin Curley', 'Louie the Cowman Beaudin') :  31
('

In [107]:
random_ballots = {**consistent_ballots, **random_inconsistent_ballots}

In [108]:
for b in random_ballots:
    if len(b) > 0:
        print(b, ": ", random_ballots[b])

('Andrew Struthers', 'Roen Hogg', 'Charles Maughan') :  2119
('Andrew Struthers', 'Charles Maughan', 'Roen Hogg') :  3722
('Andrew Struthers', 'Charles Maughan') :  511
('Andrew Struthers', 'Roen Hogg') :  279
('Andrew Struthers',) :  521
('Charles Maughan', 'Andrew Struthers', 'Roen Hogg') :  4405
('Charles Maughan', 'Andrew Struthers') :  760
('Charles Maughan', 'Roen Hogg') :  197
('Charles Maughan',) :  372
('Roen Hogg',) :  847
('Roen Hogg', 'Andrew Struthers', 'Charles Maughan') :  2484
('Roen Hogg', 'Andrew Struthers') :  228
('Roen Hogg', 'Charles Maughan') :  168
('Charles Maughan', 'Roen Hogg', 'Andrew Struthers') :  2183
('Roen Hogg', 'Charles Maughan', 'Andrew Struthers') :  1809


In [111]:
election = voting_rules(ballots, candidates)
print(election.irv())

Charles Maughan


In [112]:
consistent_election = voting_rules(consistent_ballots, candidates)
print (consistent_election.irv())

Andrew Struthers


In [114]:
random_elction = voting_rules(random_ballots, candidates)
print(random_elction.irv())

Charles Maughan
